# Demo of function easy\_export from *./swiss_utils/sdc_utilities.py*

*****

Exporting a xarray.Dataset or xarray.DataArray as netcdf or geotiff is not complicated, but requires several preparation steps which can differ depending on the type of xarray to export.

The following function (**__easy_export__**) tries to offer a one fit all solution. The user just need to call the function with the following three arguments to get netcdf or geotiff files (one per time in the xarray):
* data: xarray.Dataset or xarray.DataArray to be exported
* prfx: prefix to be used to name the exported file(s)
* ncortif: either nc(netcdf) or tif (geotiff)

As exporting xarray.Dataset can create complexe files (3 bands red, green and blue in the figure below) each of them containing "time" sub-bands (3 time period in the figure below). The easy_export function creates one file per time period and add the date and time in the filename.

![](diagrams/documentation/MS_netcdf_in_QGIS.png)

In this Jupyter script show the user can either use the in-script function (below) or import it from *./swiss_utils/sdc_utilities.py* and test it using Multi-Spectral Landsat imagery, NDVI product and DEM.

**_Geotiff are easier to load on QGIS (as netcdf loose CRS information and each bands need to be selected)_**

In [1]:
# Import necessary stuff
from datetime import datetime
import numpy as np
import xarray as xr

from utils.dc_display_map import display_map

import datacube
dc = datacube.Datacube()
api = datacube.api.API()

# IMPORT EASY_EXPORT FUNCTION
from swiss_utils.sdc_utilities import easy_export

In [2]:
# Define mini DC parameters
platform ="LANDSAT_7"
product = "ls7_ledaps_swiss"

min_lon = 7.1107
max_lon = 7.1785
min_lat = 46.4427
max_lat = 46.4777

start_date = datetime.strptime("2010-1-1", '%Y-%m-%d')
end_date = datetime.strptime("2010-2-28", '%Y-%m-%d')

In [3]:
display_map(latitude = (min_lat, max_lat), longitude = (min_lon, max_lon))

# MS bands

In [4]:
# Get a mini DC
dataset_in = dc.load(platform=platform,
                     product=product,
                     time=(start_date, end_date),
                     lon=(min_lon, max_lon), 
                     lat=(min_lat, max_lat),
                     measurements=['red','green','blue'])
print(dataset_in)

<xarray.Dataset>
Dimensions:    (latitude: 130, longitude: 251, time: 3)
Coordinates:
  * time       (time) datetime64[ns] 2010-01-12T10:07:58 2010-01-19T10:14:15 ...
  * latitude   (latitude) float64 46.48 46.48 46.48 46.48 46.48 46.48 46.48 ...
  * longitude  (longitude) float64 7.111 7.111 7.111 7.111 7.112 7.112 7.112 ...
Data variables:
    red        (time, latitude, longitude) int16 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    green      (time, latitude, longitude) int16 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    blue       (time, latitude, longitude) int16 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
Attributes:
    crs:      EPSG:4326


In [5]:
# Using the function above, it is easy...
# ...to export netcdf
easy_export(dataset_in, "MS", "nc")
# ...to export geotiff
easy_export(dataset_in, "MS", "tif")

'xarray exported succesfully'

# NDVI

In [6]:
# Get a mini DC
dataset_in = dc.load(platform=platform,
                     product=product,
                     time=(start_date, end_date),
                     lon=(min_lon, max_lon), 
                     lat=(min_lat, max_lat),
                     measurements=['red','nir'])
print(dataset_in)

<xarray.Dataset>
Dimensions:    (latitude: 130, longitude: 251, time: 3)
Coordinates:
  * time       (time) datetime64[ns] 2010-01-12T10:07:58 2010-01-19T10:14:15 ...
  * latitude   (latitude) float64 46.48 46.48 46.48 46.48 46.48 46.48 46.48 ...
  * longitude  (longitude) float64 7.111 7.111 7.111 7.111 7.112 7.112 7.112 ...
Data variables:
    red        (time, latitude, longitude) int16 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    nir        (time, latitude, longitude) int16 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
Attributes:
    crs:      EPSG:4326


In [7]:
# To get rid of division per zero warning
import warnings; warnings.simplefilter('ignore')

ndvi = (dataset_in.nir - dataset_in.red) / (dataset_in.nir + dataset_in.red)
print(ndvi)

<xarray.DataArray (time: 3, latitude: 130, longitude: 251)>
array([[[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ..., 
        [      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan]],

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ..., 
        [-0.      , -0.      , ..., -0.      , -0.      ],
        [-0.      , -0.      , ..., -0.      , -0.      ]],

       [[ 0.322835,  0.269036, ..., -0.218182, -0.300971],
        [      nan,       nan, ..., -0.325581, -0.325581],
        ..., 
        [ 0.022993,  0.012714, ...,  0.348837,  0.875   ],
        [ 0.055369,  0.048183, ...,  0.222222,  0.454545]]])
Coordinates:
  * time       (time) datetime64[ns] 2010-01-12T10:07:58 2010-01-19T10:14:15 ...
  * latitude   (latitude) float64 46.48 46.48 46.48 46.48 46.48 46.48 46.48 ...
  * longit

In [8]:
# Using the easy_export function again...
# ...to export netcdf
easy_export(ndvi, "ndvi", "nc")
# ...to export geotiff
easy_export(ndvi, "ndvi", "tif")

'xarray exported succesfully'

# DEM elevation

In [9]:
# Get a mini DC
aster = dc.load(product="terra_aster_gdm_switzerland",
                lat=(min_lat, max_lat),
                lon=(min_lon, max_lon),
                measurements=[ 'dem'])
print(aster)

<xarray.Dataset>
Dimensions:    (latitude: 130, longitude: 251, time: 1)
Coordinates:
  * time       (time) datetime64[ns] 2017-11-20T00:53:43
  * latitude   (latitude) float64 46.48 46.48 46.48 46.48 46.48 46.48 46.48 ...
  * longitude  (longitude) float64 7.111 7.111 7.111 7.111 7.112 7.112 7.112 ...
Data variables:
    dem        (time, latitude, longitude) int16 1535 1523 1510 1504 1496 ...
Attributes:
    crs:      EPSG:4326


In [10]:
# And again...
# ...to export netcdf
easy_export(aster, "dem", "nc")
# ...to export geotiff
easy_export(aster, "dem", "tif")

'xarray exported succesfully'

# DEM aspect

In [11]:
# Process apsect
# Convert to numpy.ndarray
target = aster.dem.values[0].astype(np.float32)
# Define nodata
target[target == -9999] = np.nan
# Calulate aspect (source: http://geologyandpython.com/dem-processing.html)
x, y = np.gradient(target)
aspect_rad = np.arctan2(x, y) # radians
aspect_deg = np.rad2deg(aspect_rad) # degrees
# as 0 is positioned on W (instead of N) and values range from -180 to 180
# we need to rotate the values N and 0-360
aspect_deg = np.where(aspect_deg > 90, aspect_deg - 90, aspect_deg + 270)

deg = xr.DataArray(aspect_deg,
                   coords={'latitude': aster.latitude.values,
                           'longitude': aster.longitude.values},
                   dims=('latitude', 'longitude'))
print(deg)

<xarray.DataArray (latitude: 130, longitude: 251)>
array([[ 112.619873,  111.801422,  107.525574, ...,  135.      ,  153.434967,
         161.565063],
       [ 135.      ,  136.218887,  138.57634 , ...,  137.726318,  135.      ,
         141.340195],
       [ 137.290619,  144.246124,  156.370636, ...,  163.300766,  180.      ,
         296.565063],
       ..., 
       [  90.      ,  111.801422,   96.340195, ...,  357.709381,   20.854446,
          57.264771],
       [ 111.801422,  129.805588,  118.300766, ...,   71.565048,   96.709839,
          92.60257 ],
       [ 120.96376 ,  126.869904,  138.012787, ...,  132.709396,  135.      ,
          99.462326]], dtype=float32)
Coordinates:
  * longitude  (longitude) float64 7.111 7.111 7.111 7.111 7.112 7.112 7.112 ...
  * latitude   (latitude) float64 46.48 46.48 46.48 46.48 46.48 46.48 46.48 ...


In [12]:
# And again...
# ...to export netcdf
easy_export(deg, "aspect", "nc")
# ...to export geotiff
easy_export(deg, "aspect", "tif")

'xarray exported succesfully'